In [1]:
import pandas as pd
import requests
import time
import os
from bs4 import BeautifulSoup
from zipfile import ZipFile
from io import BytesIO

In [2]:
import FinanceDataReader as fdr #pip3 install -U finance-datareader

df_krx = fdr.StockListing('KRX')

In [3]:
krx = list(df_krx['Symbol'].unique())

In [4]:
#api_key_1 = 'cb0fcabeca9860467c685d03f356d07255192809' #jeongwoo_@nate.com
#api_key_2 = '4800bd6ebc56177df6597027270cb71a1ccab416' #형민이형
#api_key_3 = '6c8e47ffc4c048c9542558809dee2924548fde6b' #jeongwoo_@hotmail.com
#api_key = '48a863e003a05fdf8afb848a434d8bba5066c731' #ganji_kjw@naver.com
#api_key_1 = 'fa56fc7ffd3c4c573f8e44fd4200a3a5b562b21c' #jungle_kim@naver.com
#api_key_2 = '44f424c3212e3473bf77559c74467b7af26b6145' #ganji_kjw@hanmail.net
#api_key_3 = '89abbb40792cb5d507673202ea1672c72b2ebcf8' #jeongwoojunglekim@gmail.com
#api_key_1 = 'bfadcf1bb1beeeab8175eeaae70dfd6a9296f508' #jeongwoo.kim@kaist.ac.kr
#api_key_1 = '762a1002ae263933f361ce7c150beb9a0c213843' #jeongwookjw@msgsafe.io
#api_key_2 = '42a08db596ec2fb94d77a49dbc0c2a14b03334aa' #slim8@snu.ac.kr
#api_key_1 = '82f02b0a5ca6a319ce289386df02f69ef716ef88' #slim8824@gmail.com
#api_key_1 = 'd100d11a0729f9dd662730ff8940063d977e9e83' #jeongwookjw1
#api_key_2 = 'a8319132e7bb608447cbfe08908363873f7ea22c' #jeongwookjw2
#api_key_3 = '9cc713f8d416074f753e2c1760a1de2022af0ee6' #jeongwookjw3
api_key_1 = '6fb4a3aa45ab0091c105349ec63bd7d5794b796c' #jeongwoo1
api_key_2 = 'e2740a12291221169f1c35a45488d7e5d281fa96' #jeongwoo2
api_key_3 = '91994e5ff42268ae9ee3518578f7ed26fa8e5d6e' #jeongwoo3
#api_key_1 = 'b9b9b4cbd79ee23c779f5f5c86f749feccefbc52' #jeongwoo4
#api_key_2 = 'cc38dc5e7e29d0f2c40aa022baa560dc7f3b2b17' #jeongwoo5
#api_key_3 = '32b87dbe27ea2a63b1e35895114c0210815e14ca' #jeongwoo6
#api_key_1 = '56c7a7e9ba7366b14b82f119f8d4755318160b46' #jeongwoo7
#api_key_2 = 'fa966992cc88d17a6c1506d9a187812096ba3aa4' #jeongwoo8
#api_key_3 = 'ae29333ab58c2ec22830f47a56763cb539df5715' #jeongwoo9

date = '20200912'

In [5]:
def get_corp_code(api_key):
    corp_dict={}
    response = requests.get('https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=' + api_key)
    zf = ZipFile(BytesIO(response.content))
    soup = BeautifulSoup(zf.open(zf.namelist()[0]).read().decode(), 'html.parser')
    for i in soup.find_all('list'):
        if i.find('stock_code').string in krx:
            corp_dict[i.find('corp_code').string] = i.find('stock_code').string
    
    return corp_dict

In [6]:
def get_all_rcpNo(api_key_1, api_key_2, api_key_3, corp_codes, date, n):
    
    business_dict={}
    banki_dict={}
    boonki_dict={}
    
    for idx, code in enumerate(corp_codes.keys()):
        if (idx>=len(corp_codes)/3*(n-1))&(idx<len(corp_codes)/3*n):
            print(str(idx-(len(corp_codes)/3*(n-1)))+' / '+str(len(corp_codes)/3)+'   '+str(idx%3), end='\r')
            i=0
            final_list = []
            if (idx%3)==0:
                api_key = api_key_1
            elif (idx%3)==1:
                api_key = api_key_2
            else:
                api_key = api_key_3
            while True:
                response = requests.get(f'https://opendart.fss.or.kr/api/list.json?crtfc_key={api_key}'
                                        f"&corp_code={code}"
                                        f'&bgn_de=19000101'
                                        f'&end_de={date}'
                                        f'&page_no={i}'
                                        f'&page_count=100')
                page = response.json()

                if page['status']=='013':
                    break

                final_list += page['list']
                if page['page_no']==page['total_page']:
                    break

                i+=1
                time.sleep(0.3)

            if page['status']!='013':
                temp = pd.DataFrame(final_list)
                business_dict[code]=list(temp[temp['report_nm'].str.contains('사업보고서')]['rcept_no'])
                banki_dict[code]=list(temp[temp['report_nm'].str.contains('반기보고서')]['rcept_no'])
                boonki_dict[code]=list(temp[temp['report_nm'].str.contains('분기보고서')]['rcept_no'])
                time.sleep(0.3)
        else:
            continue
            
    return business_dict, banki_dict, boonki_dict

In [7]:
def get_params(rcpNo, title):
    response = requests.get(f'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}')
    soup = BeautifulSoup(response.text, 'html.parser')
    
    temp = str(soup.find('head'))\
    .split(title)[1]\
    .split('cnt++')[0]\
    .split('viewDoc(')[1]\
    .split(', ')
    
    return [temp[i][1:-1] for i in range(len(temp))]

In [8]:
def get_report(params):
    response = requests.get(f'http://dart.fss.or.kr/report/viewer.do?rcpNo={params[0]}'
    f'&dcmNo={params[1]}'
    f'&eleId={params[2]}'
    f'&offset={params[3]}'
    f'&length={params[4]}'
    f'&dtd=dart3.xsd')
    soup = BeautifulSoup(response.text, 'html.parser')
    
#     for table in soup.find_all('table'):
#         table.extract()
        
    return soup.get_text(strip=True)

In [9]:
corp_codes = get_corp_code(api_key_2)

In [10]:
len(corp_codes)

2370

In [11]:
len(krx)

2571

In [12]:
res = [ ele for ele in krx ] 
for a in krx: 
    if a in list(corp_codes.values()): 
        res.remove(a) 

In [13]:
#res

In [14]:
#business_1, banki_1, boonki_1 = get_all_rcpNo(api_key_1, api_key_2, api_key_3, corp_codes, date, 1)

In [15]:
#business_2, banki_2, boonki_2 = get_all_rcpNo(api_key_1, api_key_2, api_key_3, corp_codes, date, 2)

In [16]:
business_3, banki_3, boonki_3 = get_all_rcpNo(api_key_1, api_key_2, api_key_3, corp_codes, date, 3)

In [18]:
len(business_3)

790

In [19]:
len(banki_3)

790

In [20]:
len(boonki_3)

790

In [22]:
import json
from pathlib import Path

Path('./business_3.json').write_text(json.dumps(business_3), encoding='utf-8')
Path('./banki_3.json').write_text(json.dumps(banki_3), encoding='utf-8')
Path('./boonki_3.json').write_text(json.dumps(boonki_3), encoding='utf-8')

461146

In [ ]:
frequency_category = {
    'business' : business,
    'banki' : banki,
    'boonki' : boonki
}

In [ ]:
report_category = {
    'analysis' : '이사의 경영진단 및 분석의견",',
    'protection' : '보호를 위하여 필요한 사항",'
}

In [ ]:
for name, keyword in report_category.items():
    for frequency, all_rcpNo in frequency_category.items():
        for corp_code, rcpNo_list in all_rcpNo.items():
            for rcpNo in rcpNo_list:
                try:
                    file = open(f'./reports/{name}/{frequency}/{corp_codes[corp_code]}/{rcpNo[:8]}.txt', 'w', encoding='utf-8')
                except:
                    os.makedirs(f'./reports/{name}/{frequency}/{corp_codes[corp_code]}')
                    file = open(f'./reports/{name}/{frequency}/{corp_codes[corp_code]}/{rcpNo[:8]}.txt', 'w', encoding='utf-8')
                file.write(get_report(get_params(corp_code,keyword)).replace('\xa0',''))
                file.close()

In [ ]:
asdf=100
print(str(asdf)+'23')